In [ ]:
def plot_walk_forward_analyzer_1(
    df_ohlcv=None,
    features_df=None,
    df_close_wide=None,
    df_atrp_wide=None,
    df_trp_wide=None,
    engine=None,
    universe_subset=None,  # Cascade Bridge (The missing argument)
    default_start_date="2025-01-17",
    default_lookback=10,
    default_holding=5,
    default_strategy="Sharpe (ATRP)",
    default_rank_start=1,
    default_rank_end=10,
    default_benchmark_ticker=GLOBAL_SETTINGS["benchmark_ticker"],
    master_calendar_ticker=GLOBAL_SETTINGS["calendar_ticker"],
    quality_thresholds=GLOBAL_SETTINGS["thresholds"],
    debug=True,
):

    # 1. THE PERSISTENT ENGINE GATE
    # Only build a new engine if one isn't provided
    if engine is None:
        if df_ohlcv is None:
            raise ValueError(
                "❌ No engine provided AND no df_ohlcv provided to build one."
            )
        engine = AlphaEngine(
            df_ohlcv,
            features_df=features_df,
            df_close_wide=df_close_wide,
            df_atrp_wide=df_atrp_wide,
            df_trp_wide=df_trp_wide,
            master_ticker=master_calendar_ticker,
        )

    # Initialize containers
    audit_pack = [None]  # Unified container

    # If no thresholds passed, use the global Source of Truth
    if quality_thresholds is None:
        quality_thresholds = GLOBAL_SETTINGS["thresholds"]

    # --- Widgets ---
    mode_selector = widgets.RadioButtons(
        options=["Ranking", "Manual List"],
        value="Ranking",
        description="Mode:",
        layout={"width": "max-content"},
        style={"description_width": "initial"},
    )
    lookback_input = widgets.IntText(
        value=default_lookback,
        description="Lookback (Days):",
        layout={"width": "200px"},
        style={"description_width": "initial"},
    )
    decision_date_picker = widgets.DatePicker(
        description="Decision Date:",
        value=pd.to_datetime(default_start_date),
        layout={"width": "auto"},
        style={"description_width": "initial"},
    )
    holding_input = widgets.IntText(
        value=default_holding,
        description="Holding (Days):",
        layout={"width": "200px"},
        style={"description_width": "initial"},
    )
    strategy_dropdown = widgets.Dropdown(
        options=list(METRIC_REGISTRY.keys()),
        value=default_strategy,
        description="Strategy:",
        layout={"width": "220px"},
        style={"description_width": "initial"},
    )
    benchmark_input = widgets.Text(
        value=default_benchmark_ticker,
        description="Benchmark:",
        placeholder="Enter Ticker",
        layout={"width": "180px"},
        style={"description_width": "initial"},
    )
    rank_start_input = widgets.IntText(
        value=default_rank_start,
        description="Rank Start:",
        layout={"width": "150px"},
        style={"description_width": "initial"},
    )
    rank_end_input = widgets.IntText(
        value=default_rank_end,
        description="Rank End:",
        layout={"width": "150px"},
        style={"description_width": "initial"},
    )
    manual_tickers_input = widgets.Textarea(
        value="",
        placeholder="Enter tickers...",
        description="Manual Tickers:",
        layout={"width": "400px", "height": "80px"},
        style={"description_width": "initial"},
    )
    update_button = widgets.Button(description="Run Simulation", button_style="primary")
    ticker_list_output = widgets.Output()

    # --- Layouts ---
    timeline_box = widgets.HBox(
        [lookback_input, decision_date_picker, holding_input],
        layout=widgets.Layout(
            justify_content="space-between",
            border="1px solid #ddd",
            padding="10px",
            margin="5px",
        ),
    )
    strategy_box = widgets.HBox([strategy_dropdown, benchmark_input])
    ranking_box = widgets.HBox([rank_start_input, rank_end_input])

    def on_mode_change(c):
        ranking_box.layout.display = "flex" if c["new"] == "Ranking" else "none"
        manual_tickers_input.layout.display = (
            "none" if c["new"] == "Ranking" else "flex"
        )
        strategy_dropdown.disabled = c["new"] == "Manual List"

    mode_selector.observe(on_mode_change, names="value")
    on_mode_change({"new": mode_selector.value})

    ui = widgets.VBox(
        [
            widgets.HTML(
                "<b>1. Timeline Configuration:</b> (Past <--- Decision ---> Future)"
            ),
            timeline_box,
            widgets.HTML("<b>2. Strategy Settings:</b>"),
            widgets.HBox([mode_selector, strategy_box]),
            ranking_box,
            manual_tickers_input,
            widgets.HTML("<hr>"),
            update_button,
            ticker_list_output,
        ],
        layout=widgets.Layout(margin="10px 0 20px 0"),
    )

    fig = go.FigureWidget()
    fig.update_layout(
        title="Event-Driven Walk-Forward Analysis",
        height=600,
        template="plotly_white",
        hovermode="x unified",
    )
    for i in range(50):
        fig.add_trace(go.Scatter(visible=False, line=dict(width=2)))
    fig.add_trace(
        go.Scatter(
            name="Benchmark",
            visible=True,
            line=dict(color="black", width=3, dash="dash"),
        )
    )
    fig.add_trace(
        go.Scatter(
            name="Group Portfolio", visible=True, line=dict(color="green", width=3)
        )
    )

    # --- Update Logic ---
    def update_plot(b):
        ticker_list_output.clear_output()
        manual_list = [
            t.strip().upper()
            for t in manual_tickers_input.value.split(",")
            if t.strip()
        ]
        decision_date_raw = pd.to_datetime(decision_date_picker.value)

        inputs = EngineInput(
            mode=mode_selector.value,
            start_date=decision_date_raw,
            lookback_period=lookback_input.value,
            holding_period=holding_input.value,
            metric=strategy_dropdown.value,
            benchmark_ticker=benchmark_input.value.strip().upper(),
            rank_start=rank_start_input.value,
            rank_end=rank_end_input.value,
            quality_thresholds=quality_thresholds,
            manual_tickers=manual_list,
            debug=debug,
        )

        ########################################
        with ticker_list_output:
            res = engine.run(inputs)

            # CLEANUP: audit_pack now stores the exact output object.
            # No need to manually map 'debug' or 'results' since res contains both.
            audit_pack[0] = res

            if res.error_msg:
                print(f"⚠️ Simulation Stopped: {res.error_msg}")
                return

            # ... (Plotting code only runs if res.normalized_plot_data is not empty) ...
            if not res.normalized_plot_data.empty:
                # Plotting
                with fig.batch_update():
                    cols = res.normalized_plot_data.columns.tolist()
                    for i in range(50):
                        if i < len(cols):
                            fig.data[i].update(
                                x=res.normalized_plot_data.index,
                                y=res.normalized_plot_data[cols[i]],
                                name=cols[i],
                                visible=True,
                            )
                        else:
                            fig.data[i].visible = False

                    fig.data[50].update(
                        x=res.benchmark_series.index,
                        y=res.benchmark_series.values,
                        name=f"Benchmark ({inputs.benchmark_ticker})",
                        visible=not res.benchmark_series.empty,
                    )
                    fig.data[51].update(
                        x=res.portfolio_series.index,
                        y=res.portfolio_series.values,
                        visible=True,
                    )

                    # Visual Lines
                    fig.layout.shapes = [
                        dict(
                            type="line",
                            x0=res.decision_date,
                            y0=0,
                            x1=res.decision_date,
                            y1=1,
                            xref="x",
                            yref="paper",
                            line=dict(color="red", width=2, dash="dash"),
                        ),
                        dict(
                            type="line",
                            x0=res.buy_date,
                            y0=0,
                            x1=res.buy_date,
                            y1=1,
                            xref="x",
                            yref="paper",
                            line=dict(color="blue", width=2, dash="dot"),
                        ),
                    ]

                    fig.layout.annotations = [
                        dict(
                            x=res.decision_date,
                            y=0.05,
                            xref="x",
                            yref="paper",
                            text="DECISION",
                            showarrow=False,
                            bgcolor="red",
                            font=dict(color="white"),
                        ),
                        dict(
                            x=res.buy_date,
                            y=1.0,
                            xref="x",
                            yref="paper",
                            text="ENTRY (T+1)",
                            showarrow=False,
                            bgcolor="blue",
                            font=dict(color="white"),
                        ),
                    ]

                start_date = res.start_date.date()
                act_date = res.decision_date.date()
                entry_date = res.buy_date.date()

            # Liquidity Audit Print
            if (
                inputs.mode == "Ranking"
                and res.debug_data
                and "audit_liquidity" in res.debug_data
            ):
                audit = res.debug_data["audit_liquidity"]
                if audit:
                    raw_percentile = audit.get("percentile_setting", 0)
                    keep_pct = (
                        1 - raw_percentile
                    ) * 100  # Calculates the actual portion kept
                    cut_val = audit.get("final_cutoff_usd", 0)

                    print("-" * 60)
                    print(f"🔍 LIQUIDITY CHECK (On Decision Date: {act_date})")
                    print(
                        f"   Universe Size: {audit.get('total_tickers_available')} tickers"
                    )
                    print(
                        f"   Liquidity Threshold: {raw_percentile*100:.0f}th Percentile"
                    )
                    print(f"   Action: Keeping the Top {keep_pct:.0f}% of Market")
                    print(f"   Calculated Cutoff: ${cut_val:,.0f} / day")
                    print(f"   Tickers Remaining: {audit.get('tickers_passed')}")
                    print("-" * 60)

            # --- UPDATED TIMELINE PRINT ---
            print(
                f"Timeline: Start [ {start_date} ] --> Decision [ {act_date} ] --> Cash (1d) --> Entry [ {entry_date} ] --> End [ {res.holding_end_date.date()} ]"
            )

            if inputs.mode == "Ranking":
                print(f"Ranked Tickers ({len(res.tickers)}):")
                for i in range(0, len(res.tickers), 10):
                    print(", ".join(res.tickers[i : i + 10]))
            else:
                print("Manual Portfolio Tickers:")
                for i in range(0, len(res.tickers), 10):
                    print(", ".join(res.tickers[i : i + 10]))

            m = res.perf_metrics

            # --- DRY UI GENERATION ---
            # 1. Define the metrics we want to display
            metrics_to_show = [
                ("Gain", "gain"),
                ("Sharpe", "sharpe"),
                ("Sharpe (ATRP)", "sharpe_atrp"),
                ("Sharpe (TRP)", "sharpe_trp"),  # <--- PINPOINT CHANGE: Add this line
            ]

            rows = []
            for label, key in metrics_to_show:
                p_row = {
                    "Metric": f"Group {label}",
                    "Full": m.get(f"full_p_{key}"),
                    "Lookback": m.get(f"lookback_p_{key}"),
                    "Holding": m.get(f"holding_p_{key}"),
                }
                b_row = {
                    "Metric": f"Benchmark {label}",
                    "Full": m.get(f"full_b_{key}"),
                    "Lookback": m.get(f"lookback_b_{key}"),
                    "Holding": m.get(f"holding_b_{key}"),
                }

                # Delta calculation
                d_row = {"Metric": f"== {label} Delta"}
                for col in ["Full", "Lookback", "Holding"]:
                    d_row[col] = (p_row[col] or 0) - (b_row[col] or 0)

                rows.extend([p_row, b_row, d_row])

            df_report = pd.DataFrame(rows).set_index("Metric")

            # --- 2. STYLING (The "Senior" Design) ---
            # --- 1. PREP DATA (Flattening the Index) ---
            # We convert the index to a column so "Metric" sits on the same row as other headers
            df_report = pd.DataFrame(rows)
            df_report = df_report.set_index("Metric")

            # --- 2. THE STYLING (Sleek & Proportional) ---
            def apply_sleek_style(styler):
                # Match notebook font size (usually 13px)
                styler.format("{:+.4f}", na_rep="N/A")

                # Dynamic Row Highlighting
                def row_logic(row):
                    if "Delta" in row.name:
                        return [
                            "background-color: #f9f9f9; font-weight: 600; border-top: 1px solid #ddd"
                        ] * len(row)
                    if "Group" in row.name:
                        return ["color: #2c5e8f; background-color: #fcfdfe"] * len(row)
                    return ["color: #555"] * len(
                        row
                    )  # Benchmark rows are slightly muted

                styler.apply(row_logic, axis=1)

                styler.set_table_styles(
                    [
                        # Base Table Font - Scaling down to match standard text
                        {
                            "selector": "",
                            "props": [
                                ("font-family", "inherit"),
                                ("font-size", "12px"),
                                ("border-collapse", "collapse"),
                                ("width", "auto"),
                                ("margin-left", "0"),
                            ],
                        },
                        # Header Row - Flattened and Muted
                        {
                            "selector": "th",
                            "props": [
                                ("background-color", "white"),
                                ("color", "#222"),
                                ("font-weight", "600"),
                                ("padding", "6px 12px"),
                                ("border-bottom", "2px solid #444"),
                                ("text-align", "center"),
                                (
                                    "vertical-align",
                                    "bottom",
                                ),  # Aligns 'Metric' with others
                            ],
                        },
                        # Index Column (The "Metric" labels)
                        {
                            "selector": "th.row_heading",
                            "props": [
                                ("text-align", "left"),
                                ("padding-right", "30px"),
                                ("border-bottom", "1px solid #eee"),
                            ],
                        },
                        # Cell Data - Tighter padding
                        {
                            "selector": "td",
                            "props": [
                                ("padding", "4px 12px"),
                                ("border-bottom", "1px solid #eee"),
                            ],
                        },
                        # Remove the extra "Index Name" row completely
                        {
                            "selector": "thead tr:nth-child(1) th",
                            "props": [("display", "table-cell")],
                        },
                    ]
                )

                # Hack to fix the 'Metric' alignment:
                # We remove the index name and set it as the horizontal label for the index
                styler.index.name = None

                return styler

            display(apply_sleek_style(df_report.style))

    update_button.on_click(update_plot)
    update_plot(None)
    display(ui, fig)
    return audit_pack, engine

In [ ]:
The error **`UnboundLocalError: quality_thresholds`** happened because in the refactored signature, `quality_thresholds` was removed from the function arguments but the code inside still tried to check `if quality_thresholds is None:`.

To fix this and make the function fully "RL-Ready" and "Cascade-Ready," we need to update the signature to include both the persistent engine and the quality thresholds.

### The Corrected `plot_walk_forward_analyzer` Signature

Replace the top part of your `plot_walk_forward_analyzer` function with this. This version correctly handles the "Persistent Engine" and the "Subsetting" logic.

```python
def plot_walk_forward_analyzer(
    df_ohlcv=None,
    features_df=None,
    df_close_wide=None,
    df_atrp_wide=None,
    df_trp_wide=None,
    engine=None,            # <--- ADDED: Support persistent engine
    universe_subset=None,   # <--- ADDED: Support cascade filtering
    quality_thresholds=None, # <--- FIXED: Restore this to signature
    **kwargs                 # Captures default_start_date, etc.
):
    # 1. PERSISTENT ENGINE LOGIC
    if engine is None:
        # If no engine provided, build one. If provided, we use the one passed in.
        engine = AlphaEngine(
            df_ohlcv,
            features_df=features_df,
            df_close_wide=df_close_wide,
            df_atrp_wide=df_atrp_wide,
            df_trp_wide=df_trp_wide,
        )

    # 2. INITIALIZE CONTAINERS
    audit_pack = [None] 

    # 3. THRESHOLD LOGIC (Now safely works)
    if quality_thresholds is None:
        quality_thresholds = GLOBAL_SETTINGS["thresholds"]

    # --- Widget Default Extraction from kwargs ---
    # This ensures your 'default_start_date' etc. actually get used
    d_start = kwargs.get("default_start_date", "2025-01-17")
    d_lookback = kwargs.get("default_lookback", 10)
    d_holding = kwargs.get("default_holding", 5)
    d_strategy = kwargs.get("default_strategy", "Sharpe (ATRP)")
    d_rank_end = kwargs.get("default_rank_end", 10)
    d_debug = kwargs.get("debug", True)

    # ... (Widget definitions using the 'd_' variables above) ...
    # Example:
    # lookback_input = widgets.IntText(value=d_lookback, ...)

    # --- Inside the update_plot(b) function: ---
    def update_plot(b):
        # ... (Date logic) ...
        
        inputs = EngineInput(
            mode=mode_selector.value,
            start_date=decision_date_raw,
            lookback_period=lookback_input.value,
            holding_period=holding_input.value,
            metric=strategy_dropdown.value,
            benchmark_ticker=benchmark_input.value.strip().upper(),
            rank_start=rank_start_input.value,
            rank_end=rank_end_input.value,
            quality_thresholds=quality_thresholds,
            manual_tickers=manual_list,
            universe_subset=universe_subset, # <--- PASS THE SUBSET TO ENGINE
            debug=d_debug,
        )

        with ticker_list_output:
            res = engine.run(inputs)
            audit_pack[0] = res
            # ... (Rest of plotting logic) ...

    # ... (Rest of UI Layout code) ...

    # MUST RETURN BOTH
    return audit_pack, engine 
```

### Why this works:
1.  **Fixed Scope:** By putting `quality_thresholds=None` in the signature, Python no longer treats it as an "unbound" local variable. It is a defined argument that defaults to the global settings if you don't pass anything.
2.  **Flexible Defaults:** Using `kwargs.get()` allows you to pass parameters like `default_strategy="Sharpe (TRP)"` and have them actually show up in the UI widgets when the cell runs.
3.  **The "Bridge":** The `universe_subset=universe_subset` line inside the `update_plot` is the actual "Sniper Scope." It tells the engine to ignore the 3,000 stocks in the world and only focus on the ones provided.

### The Cleanup Step:
Now you can run your cells exactly as you intended:
1.  **Cell 1:** Create `master_engine`.
2.  **Cell 2:** `audit_1, _ = plot_walk_forward_analyzer(engine=master_engine, ...)`
3.  **Cell 3:** `audit_2, _ = plot_walk_forward_analyzer(engine=master_engine, universe_subset=audit_1[0].tickers, ...)`

This structure is now perfectly optimized for your future RL Agent to take control.

In [ ]:
def plot_walk_forward_analyzer(
    df_ohlcv=None,
    features_df=None,
    df_close_wide=None,
    df_atrp_wide=None,
    df_trp_wide=None,
    engine=None,  # Support for persistent engine
    universe_subset=None,  # Support for cascade filtering
    quality_thresholds=None,
    master_calendar_ticker=GLOBAL_SETTINGS["calendar_ticker"],
    **kwargs,
):
    # 1. PERSISTENT ENGINE LOGIC
    if engine is None:
        engine = AlphaEngine(
            df_ohlcv,
            features_df=features_df,
            df_close_wide=df_close_wide,
            df_atrp_wide=df_atrp_wide,
            df_trp_wide=df_trp_wide,
            master_ticker=master_calendar_ticker,
        )

    # 2. CONTAINERS & THRESHOLDS
    audit_pack = [None]
    if quality_thresholds is None:
        quality_thresholds = GLOBAL_SETTINGS["thresholds"]

    # 3. EXTRACT DEFAULTS FROM KWARGS
    d_start = kwargs.get("default_start_date", "2025-01-17")
    d_lookback = kwargs.get("default_lookback", 10)
    d_holding = kwargs.get("default_holding", 5)
    d_strategy = kwargs.get("default_strategy", "Sharpe (ATRP)")
    d_rank_start = kwargs.get("default_rank_start", 1)  # Fixed: Was missing
    d_rank_end = kwargs.get("default_rank_end", 10)
    d_benchmark = kwargs.get(
        "default_benchmark_ticker", GLOBAL_SETTINGS["benchmark_ticker"]
    )
    d_debug = kwargs.get("debug", True)  # Target for RL/Audit gating

    # --- WIDGETS ---
    mode_selector = widgets.RadioButtons(
        options=["Ranking", "Manual List"],
        value="Ranking",
        description="Mode:",
        layout={"width": "max-content"},
        style={"description_width": "initial"},
    )
    lookback_input = widgets.IntText(
        value=d_lookback,
        description="Lookback (Days):",
        layout={"width": "200px"},
        style={"description_width": "initial"},
    )
    decision_date_picker = widgets.DatePicker(
        description="Decision Date:",
        value=pd.to_datetime(d_start),
        layout={"width": "auto"},
        style={"description_width": "initial"},
    )
    holding_input = widgets.IntText(
        value=d_holding,
        description="Holding (Days):",
        layout={"width": "200px"},
        style={"description_width": "initial"},
    )
    strategy_dropdown = widgets.Dropdown(
        options=list(METRIC_REGISTRY.keys()),
        value=d_strategy,
        description="Strategy:",
        layout={"width": "220px"},
        style={"description_width": "initial"},
    )
    benchmark_input = widgets.Text(
        value=d_benchmark,
        description="Benchmark:",
        placeholder="Enter Ticker",
        layout={"width": "180px"},
        style={"description_width": "initial"},
    )
    rank_start_input = widgets.IntText(
        value=d_rank_start,
        description="Rank Start:",
        layout={"width": "150px"},
        style={"description_width": "initial"},
    )
    rank_end_input = widgets.IntText(
        value=d_rank_end,
        description="Rank End:",
        layout={"width": "150px"},
        style={"description_width": "initial"},
    )
    manual_tickers_input = widgets.Textarea(
        value="",
        placeholder="Enter tickers...",
        description="Manual Tickers:",
        layout={"width": "400px", "height": "80px"},
        style={"description_width": "initial"},
    )
    update_button = widgets.Button(description="Run Simulation", button_style="primary")
    ticker_list_output = widgets.Output()

    # --- LAYOUTS ---
    timeline_box = widgets.HBox(
        [lookback_input, decision_date_picker, holding_input],
        layout=widgets.Layout(
            justify_content="space-between",
            border="1px solid #ddd",
            padding="10px",
            margin="5px",
        ),
    )
    strategy_box = widgets.HBox([strategy_dropdown, benchmark_input])
    ranking_box = widgets.HBox([rank_start_input, rank_end_input])

    def on_mode_change(c):
        ranking_box.layout.display = "flex" if c["new"] == "Ranking" else "none"
        manual_tickers_input.layout.display = (
            "none" if c["new"] == "Ranking" else "flex"
        )
        strategy_dropdown.disabled = c["new"] == "Manual List"

    mode_selector.observe(on_mode_change, names="value")
    on_mode_change({"new": mode_selector.value})

    ui = widgets.VBox(
        [
            widgets.HTML(
                "<b>1. Timeline Configuration:</b> (Past <--- Decision ---> Future)"
            ),
            timeline_box,
            widgets.HTML("<b>2. Strategy Settings:</b>"),
            widgets.HBox([mode_selector, strategy_box]),
            ranking_box,
            manual_tickers_input,
            widgets.HTML("<hr>"),
            update_button,
            ticker_list_output,
        ],
        layout=widgets.Layout(margin="10px 0 20px 0"),
    )

    fig = go.FigureWidget()
    fig.update_layout(
        title="Event-Driven Walk-Forward Analysis",
        height=600,
        template="plotly_white",
        hovermode="x unified",
    )
    for i in range(50):
        fig.add_trace(go.Scatter(visible=False, line=dict(width=2)))
    fig.add_trace(
        go.Scatter(
            name="Benchmark",
            visible=True,
            line=dict(color="black", width=3, dash="dash"),
        )
    )
    fig.add_trace(
        go.Scatter(
            name="Group Portfolio", visible=True, line=dict(color="green", width=3)
        )
    )

    # --- UPDATE LOGIC ---
    def update_plot(b):
        ticker_list_output.clear_output()
        manual_list = [
            t.strip().upper()
            for t in manual_tickers_input.value.split(",")
            if t.strip()
        ]
        decision_date_raw = pd.to_datetime(decision_date_picker.value)

        inputs = EngineInput(
            mode=mode_selector.value,
            start_date=decision_date_raw,
            lookback_period=lookback_input.value,
            holding_period=holding_input.value,
            metric=strategy_dropdown.value,
            benchmark_ticker=benchmark_input.value.strip().upper(),
            rank_start=rank_start_input.value,
            rank_end=rank_end_input.value,
            quality_thresholds=quality_thresholds,
            manual_tickers=manual_list,
            universe_subset=universe_subset,
            debug=d_debug,  # Fixed: Now correctly using kwarg debug flag
        )

        with ticker_list_output:
            res = engine.run(inputs)
            audit_pack[0] = res

            if res.error_msg:
                print(f"⚠️ Simulation Stopped: {res.error_msg}")
                return

            if not res.normalized_plot_data.empty:
                with fig.batch_update():
                    cols = res.normalized_plot_data.columns.tolist()
                    for i in range(50):
                        if i < len(cols):
                            fig.data[i].update(
                                x=res.normalized_plot_data.index,
                                y=res.normalized_plot_data[cols[i]],
                                name=cols[i],
                                visible=True,
                            )
                        else:
                            fig.data[i].visible = False

                    fig.data[50].update(
                        x=res.benchmark_series.index,
                        y=res.benchmark_series.values,
                        name=f"Benchmark ({inputs.benchmark_ticker})",
                        visible=not res.benchmark_series.empty,
                    )
                    fig.data[51].update(
                        x=res.portfolio_series.index,
                        y=res.portfolio_series.values,
                        visible=True,
                    )

                    fig.layout.shapes = [
                        dict(
                            type="line",
                            x0=res.decision_date,
                            y0=0,
                            x1=res.decision_date,
                            y1=1,
                            xref="x",
                            yref="paper",
                            line=dict(color="red", width=2, dash="dash"),
                        ),
                        dict(
                            type="line",
                            x0=res.buy_date,
                            y0=0,
                            x1=res.buy_date,
                            y1=1,
                            xref="x",
                            yref="paper",
                            line=dict(color="blue", width=2, dash="dot"),
                        ),
                    ]
                    fig.layout.annotations = [
                        dict(
                            x=res.decision_date,
                            y=0.05,
                            xref="x",
                            yref="paper",
                            text="DECISION",
                            showarrow=False,
                            bgcolor="red",
                            font=dict(color="white"),
                        ),
                        dict(
                            x=res.buy_date,
                            y=1.0,
                            xref="x",
                            yref="paper",
                            text="ENTRY (T+1)",
                            showarrow=False,
                            bgcolor="blue",
                            font=dict(color="white"),
                        ),
                    ]

            # --- LIQUIDITY AUDIT PRINT ---
            if (
                inputs.mode == "Ranking"
                and res.debug_data
                and "audit_liquidity" in res.debug_data
            ):
                audit = res.debug_data["audit_liquidity"]
                if "final_cutoff_usd" in audit:
                    raw_pct = audit.get("percentile_setting", 0)
                    print("-" * 60)
                    print(
                        f"🔍 LIQUIDITY CHECK (On Decision Date: {res.decision_date.date()})"
                    )
                    print(
                        f"   Universe Size: {audit.get('total_tickers_available')} tickers"
                    )
                    print(f"   Liquidity Threshold: {raw_pct*100:.0f}th Percentile")
                    print(
                        f"   Calculated Cutoff: ${audit.get('final_cutoff_usd'):,.0f} / day"
                    )
                    print(f"   Tickers Remaining: {audit.get('tickers_passed')}")
                    print("-" * 60)

            print(
                f"Timeline: Start [{res.start_date.date()}] --> Decision [{res.decision_date.date()}] --> Entry [{res.buy_date.date()}] --> End [{res.holding_end_date.date()}]"
            )
            print(
                f"{'Ranked' if inputs.mode == 'Ranking' else 'Manual'} Tickers ({len(res.tickers)}):"
            )
            for i in range(0, len(res.tickers), 10):
                print(", ".join(res.tickers[i : i + 10]))

            # --- REPORTING ---
            metrics_to_show = [
                ("Gain", "gain"),
                ("Sharpe", "sharpe"),
                ("Sharpe (ATRP)", "sharpe_atrp"),
                ("Sharpe (TRP)", "sharpe_trp"),
            ]
            rows = []
            for label, key in metrics_to_show:
                p_row = {
                    "Metric": f"Group {label}",
                    "Full": res.perf_metrics.get(f"full_p_{key}"),
                    "Lookback": res.perf_metrics.get(f"lookback_p_{key}"),
                    "Holding": res.perf_metrics.get(f"holding_p_{key}"),
                }
                b_row = {
                    "Metric": f"Benchmark {label}",
                    "Full": res.perf_metrics.get(f"full_b_{key}"),
                    "Lookback": res.perf_metrics.get(f"lookback_b_{key}"),
                    "Holding": res.perf_metrics.get(f"holding_b_{key}"),
                }
                d_row = {
                    "Metric": f"== {label} Delta",
                    "Full": (p_row["Full"] or 0) - (b_row["Full"] or 0),
                    "Lookback": (p_row["Lookback"] or 0) - (b_row["Lookback"] or 0),
                    "Holding": (p_row["Holding"] or 0) - (b_row["Holding"] or 0),
                }
                rows.extend([p_row, b_row, d_row])

            df_report = pd.DataFrame(rows).set_index("Metric")

            def apply_sleek_style(styler):
                styler.format("{:+.4f}", na_rep="N/A")
                styler.apply(
                    lambda row: [
                        (
                            "background-color: #f9f9f9; font-weight: 600; border-top: 1px solid #ddd"
                            if "Delta" in row.name
                            else "color: #2c5e8f" if "Group" in row.name else ""
                        )
                        for _ in row
                    ],
                    axis=1,
                )
                styler.set_table_styles(
                    [
                        {
                            "selector": "th",
                            "props": [
                                ("background-color", "white"),
                                ("color", "#222"),
                                ("font-weight", "600"),
                                ("padding", "6px 12px"),
                                ("border-bottom", "2px solid #444"),
                            ],
                        },
                        {
                            "selector": "td",
                            "props": [
                                ("padding", "4px 12px"),
                                ("border-bottom", "1px solid #eee"),
                            ],
                        },
                    ]
                )
                styler.index.name = None
                return styler

            display(apply_sleek_style(df_report.style))

    update_button.on_click(update_plot)
    update_plot(None)
    display(ui, fig)
    return audit_pack, engine

In [ ]:
def plot_walk_forward_analyzer(
    df_ohlcv=None,
    features_df=None,
    df_close_wide=None,
    df_atrp_wide=None,
    df_trp_wide=None,
    engine=None,  # <--- ADDED: Support persistent engine
    universe_subset=None,  # <--- ADDED: Support cascade filtering
    quality_thresholds=None,  # <--- FIXED: Restore this to signature
    **kwargs,  # Captures default_start_date, etc.
):
    # 1. PERSISTENT ENGINE LOGIC
    # If no engine provided, build one (Coarse Run). If provided, use it (Sniper Run).
    if engine is None:
        engine = AlphaEngine(
            df_ohlcv,
            features_df=features_df,
            df_close_wide=df_close_wide,
            df_atrp_wide=df_atrp_wide,
            df_trp_wide=df_trp_wide,
        )

    # Initialize containers
    audit_pack = [None]  # Unified container

    # 3. THRESHOLD LOGIC (Now safely works)
    if quality_thresholds is None:
        quality_thresholds = GLOBAL_SETTINGS["thresholds"]

    # --- Widget Default Extraction from kwargs ---
    # This ensures your 'default_start_date' etc. actually get used
    d_start = kwargs.get("default_start_date", "2025-01-17")
    d_lookback = kwargs.get("default_lookback", 10)
    d_holding = kwargs.get("default_holding", 5)
    d_strategy = kwargs.get("default_strategy", "Sharpe (ATRP)")
    d_rank_end = kwargs.get("default_rank_end", 10)
    d_debug = kwargs.get("debug", True)

    # --- Widgets ---
    mode_selector = widgets.RadioButtons(
        options=["Ranking", "Manual List"],
        value="Ranking",
        description="Mode:",
        layout={"width": "max-content"},
        style={"description_width": "initial"},
    )
    lookback_input = widgets.IntText(
        value=d_lookback,
        description="Lookback (Days):",
        layout={"width": "200px"},
        style={"description_width": "initial"},
    )
    decision_date_picker = widgets.DatePicker(
        description="Decision Date:",
        value=pd.to_datetime(d_start),
        layout={"width": "auto"},
        style={"description_width": "initial"},
    )
    holding_input = widgets.IntText(
        value=d_holding,
        description="Holding (Days):",
        layout={"width": "200px"},
        style={"description_width": "initial"},
    )
    strategy_dropdown = widgets.Dropdown(
        options=list(METRIC_REGISTRY.keys()),
        value=d_strategy,
        description="Strategy:",
        layout={"width": "220px"},
        style={"description_width": "initial"},
    )
    benchmark_input = widgets.Text(
        value=default_benchmark_ticker,
        description="Benchmark:",
        placeholder="Enter Ticker",
        layout={"width": "180px"},
        style={"description_width": "initial"},
    )
    rank_start_input = widgets.IntText(
        value=default_rank_start,
        description="Rank Start:",
        layout={"width": "150px"},
        style={"description_width": "initial"},
    )
    rank_end_input = widgets.IntText(
        value=d_rank_end,
        description="Rank End:",
        layout={"width": "150px"},
        style={"description_width": "initial"},
    )
    manual_tickers_input = widgets.Textarea(
        value="",
        placeholder="Enter tickers...",
        description="Manual Tickers:",
        layout={"width": "400px", "height": "80px"},
        style={"description_width": "initial"},
    )
    update_button = widgets.Button(description="Run Simulation", button_style="primary")
    ticker_list_output = widgets.Output()

    # --- Layouts ---
    timeline_box = widgets.HBox(
        [lookback_input, decision_date_picker, holding_input],
        layout=widgets.Layout(
            justify_content="space-between",
            border="1px solid #ddd",
            padding="10px",
            margin="5px",
        ),
    )
    strategy_box = widgets.HBox([strategy_dropdown, benchmark_input])
    ranking_box = widgets.HBox([rank_start_input, rank_end_input])

    def on_mode_change(c):
        ranking_box.layout.display = "flex" if c["new"] == "Ranking" else "none"
        manual_tickers_input.layout.display = (
            "none" if c["new"] == "Ranking" else "flex"
        )
        strategy_dropdown.disabled = c["new"] == "Manual List"

    mode_selector.observe(on_mode_change, names="value")
    on_mode_change({"new": mode_selector.value})

    ui = widgets.VBox(
        [
            widgets.HTML(
                "<b>1. Timeline Configuration:</b> (Past <--- Decision ---> Future)"
            ),
            timeline_box,
            widgets.HTML("<b>2. Strategy Settings:</b>"),
            widgets.HBox([mode_selector, strategy_box]),
            ranking_box,
            manual_tickers_input,
            widgets.HTML("<hr>"),
            update_button,
            ticker_list_output,
        ],
        layout=widgets.Layout(margin="10px 0 20px 0"),
    )

    fig = go.FigureWidget()
    fig.update_layout(
        title="Event-Driven Walk-Forward Analysis",
        height=600,
        template="plotly_white",
        hovermode="x unified",
    )
    for i in range(50):
        fig.add_trace(go.Scatter(visible=False, line=dict(width=2)))
    fig.add_trace(
        go.Scatter(
            name="Benchmark",
            visible=True,
            line=dict(color="black", width=3, dash="dash"),
        )
    )
    fig.add_trace(
        go.Scatter(
            name="Group Portfolio", visible=True, line=dict(color="green", width=3)
        )
    )

    # --- Update Logic ---
    def update_plot(b):
        ticker_list_output.clear_output()
        manual_list = [
            t.strip().upper()
            for t in manual_tickers_input.value.split(",")
            if t.strip()
        ]
        decision_date_raw = pd.to_datetime(decision_date_picker.value)

        inputs = EngineInput(
            mode=mode_selector.value,
            start_date=decision_date_raw,
            lookback_period=lookback_input.value,
            holding_period=holding_input.value,
            metric=strategy_dropdown.value,
            benchmark_ticker=benchmark_input.value.strip().upper(),
            rank_start=rank_start_input.value,
            rank_end=rank_end_input.value,
            quality_thresholds=quality_thresholds,
            manual_tickers=manual_list,
            universe_subset=universe_subset,  # <--- PASS SUBSET TO ENGINE
            debug=debug,
        )

        ########################################
        with ticker_list_output:
            res = engine.run(inputs)

            # CLEANUP: audit_pack now stores the exact output object.
            # No need to manually map 'debug' or 'results' since res contains both.
            audit_pack[0] = res

            if res.error_msg:
                print(f"⚠️ Simulation Stopped: {res.error_msg}")
                return

            # ... (Plotting code only runs if res.normalized_plot_data is not empty) ...
            if not res.normalized_plot_data.empty:
                # Plotting
                with fig.batch_update():
                    cols = res.normalized_plot_data.columns.tolist()
                    for i in range(50):
                        if i < len(cols):
                            fig.data[i].update(
                                x=res.normalized_plot_data.index,
                                y=res.normalized_plot_data[cols[i]],
                                name=cols[i],
                                visible=True,
                            )
                        else:
                            fig.data[i].visible = False

                    fig.data[50].update(
                        x=res.benchmark_series.index,
                        y=res.benchmark_series.values,
                        name=f"Benchmark ({inputs.benchmark_ticker})",
                        visible=not res.benchmark_series.empty,
                    )
                    fig.data[51].update(
                        x=res.portfolio_series.index,
                        y=res.portfolio_series.values,
                        visible=True,
                    )

                    # Visual Lines
                    fig.layout.shapes = [
                        dict(
                            type="line",
                            x0=res.decision_date,
                            y0=0,
                            x1=res.decision_date,
                            y1=1,
                            xref="x",
                            yref="paper",
                            line=dict(color="red", width=2, dash="dash"),
                        ),
                        dict(
                            type="line",
                            x0=res.buy_date,
                            y0=0,
                            x1=res.buy_date,
                            y1=1,
                            xref="x",
                            yref="paper",
                            line=dict(color="blue", width=2, dash="dot"),
                        ),
                    ]

                    fig.layout.annotations = [
                        dict(
                            x=res.decision_date,
                            y=0.05,
                            xref="x",
                            yref="paper",
                            text="DECISION",
                            showarrow=False,
                            bgcolor="red",
                            font=dict(color="white"),
                        ),
                        dict(
                            x=res.buy_date,
                            y=1.0,
                            xref="x",
                            yref="paper",
                            text="ENTRY (T+1)",
                            showarrow=False,
                            bgcolor="blue",
                            font=dict(color="white"),
                        ),
                    ]

                start_date = res.start_date.date()
                act_date = res.decision_date.date()
                entry_date = res.buy_date.date()

            # Liquidity Audit Print
            if (
                inputs.mode == "Ranking"
                and res.debug_data
                and "audit_liquidity" in res.debug_data
            ):
                audit = res.debug_data["audit_liquidity"]
                if audit:
                    raw_percentile = audit.get("percentile_setting", 0)
                    keep_pct = (
                        1 - raw_percentile
                    ) * 100  # Calculates the actual portion kept
                    cut_val = audit.get("final_cutoff_usd", 0)

                    print("-" * 60)
                    print(f"🔍 LIQUIDITY CHECK (On Decision Date: {act_date})")
                    print(
                        f"   Universe Size: {audit.get('total_tickers_available')} tickers"
                    )
                    print(
                        f"   Liquidity Threshold: {raw_percentile*100:.0f}th Percentile"
                    )
                    print(f"   Action: Keeping the Top {keep_pct:.0f}% of Market")
                    print(f"   Calculated Cutoff: ${cut_val:,.0f} / day")
                    print(f"   Tickers Remaining: {audit.get('tickers_passed')}")
                    print("-" * 60)

            # --- UPDATED TIMELINE PRINT ---
            print(
                f"Timeline: Start [ {start_date} ] --> Decision [ {act_date} ] --> Cash (1d) --> Entry [ {entry_date} ] --> End [ {res.holding_end_date.date()} ]"
            )

            if inputs.mode == "Ranking":
                print(f"Ranked Tickers ({len(res.tickers)}):")
                for i in range(0, len(res.tickers), 10):
                    print(", ".join(res.tickers[i : i + 10]))
            else:
                print("Manual Portfolio Tickers:")
                for i in range(0, len(res.tickers), 10):
                    print(", ".join(res.tickers[i : i + 10]))

            m = res.perf_metrics

            # --- DRY UI GENERATION ---
            # 1. Define the metrics we want to display
            metrics_to_show = [
                ("Gain", "gain"),
                ("Sharpe", "sharpe"),
                ("Sharpe (ATRP)", "sharpe_atrp"),
                ("Sharpe (TRP)", "sharpe_trp"),  # <--- PINPOINT CHANGE: Add this line
            ]

            rows = []
            for label, key in metrics_to_show:
                p_row = {
                    "Metric": f"Group {label}",
                    "Full": m.get(f"full_p_{key}"),
                    "Lookback": m.get(f"lookback_p_{key}"),
                    "Holding": m.get(f"holding_p_{key}"),
                }
                b_row = {
                    "Metric": f"Benchmark {label}",
                    "Full": m.get(f"full_b_{key}"),
                    "Lookback": m.get(f"lookback_b_{key}"),
                    "Holding": m.get(f"holding_b_{key}"),
                }

                # Delta calculation
                d_row = {"Metric": f"== {label} Delta"}
                for col in ["Full", "Lookback", "Holding"]:
                    d_row[col] = (p_row[col] or 0) - (b_row[col] or 0)

                rows.extend([p_row, b_row, d_row])

            df_report = pd.DataFrame(rows).set_index("Metric")

            # --- 2. STYLING (The "Senior" Design) ---
            # --- 1. PREP DATA (Flattening the Index) ---
            # We convert the index to a column so "Metric" sits on the same row as other headers
            df_report = pd.DataFrame(rows)
            df_report = df_report.set_index("Metric")

            # --- 2. THE STYLING (Sleek & Proportional) ---
            def apply_sleek_style(styler):
                # Match notebook font size (usually 13px)
                styler.format("{:+.4f}", na_rep="N/A")

                # Dynamic Row Highlighting
                def row_logic(row):
                    if "Delta" in row.name:
                        return [
                            "background-color: #f9f9f9; font-weight: 600; border-top: 1px solid #ddd"
                        ] * len(row)
                    if "Group" in row.name:
                        return ["color: #2c5e8f; background-color: #fcfdfe"] * len(row)
                    return ["color: #555"] * len(
                        row
                    )  # Benchmark rows are slightly muted

                styler.apply(row_logic, axis=1)

                styler.set_table_styles(
                    [
                        # Base Table Font - Scaling down to match standard text
                        {
                            "selector": "",
                            "props": [
                                ("font-family", "inherit"),
                                ("font-size", "12px"),
                                ("border-collapse", "collapse"),
                                ("width", "auto"),
                                ("margin-left", "0"),
                            ],
                        },
                        # Header Row - Flattened and Muted
                        {
                            "selector": "th",
                            "props": [
                                ("background-color", "white"),
                                ("color", "#222"),
                                ("font-weight", "600"),
                                ("padding", "6px 12px"),
                                ("border-bottom", "2px solid #444"),
                                ("text-align", "center"),
                                (
                                    "vertical-align",
                                    "bottom",
                                ),  # Aligns 'Metric' with others
                            ],
                        },
                        # Index Column (The "Metric" labels)
                        {
                            "selector": "th.row_heading",
                            "props": [
                                ("text-align", "left"),
                                ("padding-right", "30px"),
                                ("border-bottom", "1px solid #eee"),
                            ],
                        },
                        # Cell Data - Tighter padding
                        {
                            "selector": "td",
                            "props": [
                                ("padding", "4px 12px"),
                                ("border-bottom", "1px solid #eee"),
                            ],
                        },
                        # Remove the extra "Index Name" row completely
                        {
                            "selector": "thead tr:nth-child(1) th",
                            "props": [("display", "table-cell")],
                        },
                    ]
                )

                # Hack to fix the 'Metric' alignment:
                # We remove the index name and set it as the horizontal label for the index
                styler.index.name = None

                return styler

            display(apply_sleek_style(df_report.style))

    update_button.on_click(update_plot)
    update_plot(None)
    display(ui, fig)
    return audit_pack, engine

In [ ]:
def plot_walk_forward_analyzer(
    df_ohlcv=None,
    features_df=None,
    df_close_wide=None,
    df_atrp_wide=None,
    df_trp_wide=None,
    engine=None,  # <--- NEW: Accept existing engine
    universe_subset=None,  # <--- NEW: Accept subset from previous run
    **kwargs,  # (default_start_date, etc.)
):
    # 1. PERSISTENT ENGINE LOGIC
    # If no engine provided, build one (Coarse Run). If provided, use it (Sniper Run).
    if engine is None:
        engine = AlphaEngine(
            df_ohlcv,
            features_df=features_df,
            df_close_wide=df_close_wide,
            df_atrp_wide=df_atrp_wide,
            df_trp_wide=df_trp_wide,
        )

    # Initialize containers
    audit_pack = [None]  # Unified container

    # If no thresholds passed, use the global Source of Truth
    if quality_thresholds is None:
        quality_thresholds = GLOBAL_SETTINGS["thresholds"]

    # --- Widgets ---
    mode_selector = widgets.RadioButtons(
        options=["Ranking", "Manual List"],
        value="Ranking",
        description="Mode:",
        layout={"width": "max-content"},
        style={"description_width": "initial"},
    )
    lookback_input = widgets.IntText(
        value=default_lookback,
        description="Lookback (Days):",
        layout={"width": "200px"},
        style={"description_width": "initial"},
    )
    decision_date_picker = widgets.DatePicker(
        description="Decision Date:",
        value=pd.to_datetime(default_start_date),
        layout={"width": "auto"},
        style={"description_width": "initial"},
    )
    holding_input = widgets.IntText(
        value=default_holding,
        description="Holding (Days):",
        layout={"width": "200px"},
        style={"description_width": "initial"},
    )
    strategy_dropdown = widgets.Dropdown(
        options=list(METRIC_REGISTRY.keys()),
        value=default_strategy,
        description="Strategy:",
        layout={"width": "220px"},
        style={"description_width": "initial"},
    )
    benchmark_input = widgets.Text(
        value=default_benchmark_ticker,
        description="Benchmark:",
        placeholder="Enter Ticker",
        layout={"width": "180px"},
        style={"description_width": "initial"},
    )
    rank_start_input = widgets.IntText(
        value=default_rank_start,
        description="Rank Start:",
        layout={"width": "150px"},
        style={"description_width": "initial"},
    )
    rank_end_input = widgets.IntText(
        value=default_rank_end,
        description="Rank End:",
        layout={"width": "150px"},
        style={"description_width": "initial"},
    )
    manual_tickers_input = widgets.Textarea(
        value="",
        placeholder="Enter tickers...",
        description="Manual Tickers:",
        layout={"width": "400px", "height": "80px"},
        style={"description_width": "initial"},
    )
    update_button = widgets.Button(description="Run Simulation", button_style="primary")
    ticker_list_output = widgets.Output()

    # --- Layouts ---
    timeline_box = widgets.HBox(
        [lookback_input, decision_date_picker, holding_input],
        layout=widgets.Layout(
            justify_content="space-between",
            border="1px solid #ddd",
            padding="10px",
            margin="5px",
        ),
    )
    strategy_box = widgets.HBox([strategy_dropdown, benchmark_input])
    ranking_box = widgets.HBox([rank_start_input, rank_end_input])

    def on_mode_change(c):
        ranking_box.layout.display = "flex" if c["new"] == "Ranking" else "none"
        manual_tickers_input.layout.display = (
            "none" if c["new"] == "Ranking" else "flex"
        )
        strategy_dropdown.disabled = c["new"] == "Manual List"

    mode_selector.observe(on_mode_change, names="value")
    on_mode_change({"new": mode_selector.value})

    ui = widgets.VBox(
        [
            widgets.HTML(
                "<b>1. Timeline Configuration:</b> (Past <--- Decision ---> Future)"
            ),
            timeline_box,
            widgets.HTML("<b>2. Strategy Settings:</b>"),
            widgets.HBox([mode_selector, strategy_box]),
            ranking_box,
            manual_tickers_input,
            widgets.HTML("<hr>"),
            update_button,
            ticker_list_output,
        ],
        layout=widgets.Layout(margin="10px 0 20px 0"),
    )

    fig = go.FigureWidget()
    fig.update_layout(
        title="Event-Driven Walk-Forward Analysis",
        height=600,
        template="plotly_white",
        hovermode="x unified",
    )
    for i in range(50):
        fig.add_trace(go.Scatter(visible=False, line=dict(width=2)))
    fig.add_trace(
        go.Scatter(
            name="Benchmark",
            visible=True,
            line=dict(color="black", width=3, dash="dash"),
        )
    )
    fig.add_trace(
        go.Scatter(
            name="Group Portfolio", visible=True, line=dict(color="green", width=3)
        )
    )

    # --- Update Logic ---
    def update_plot(b):
        ticker_list_output.clear_output()
        manual_list = [
            t.strip().upper()
            for t in manual_tickers_input.value.split(",")
            if t.strip()
        ]
        decision_date_raw = pd.to_datetime(decision_date_picker.value)

        inputs = EngineInput(
            mode=mode_selector.value,
            start_date=decision_date_raw,
            lookback_period=lookback_input.value,
            holding_period=holding_input.value,
            metric=strategy_dropdown.value,
            benchmark_ticker=benchmark_input.value.strip().upper(),
            rank_start=rank_start_input.value,
            rank_end=rank_end_input.value,
            quality_thresholds=quality_thresholds,
            manual_tickers=manual_list,
            universe_subset=universe_subset,  # <--- PASS SUBSET TO ENGINE
            debug=debug,
        )

        ########################################
        with ticker_list_output:
            res = engine.run(inputs)

            # CLEANUP: audit_pack now stores the exact output object.
            # No need to manually map 'debug' or 'results' since res contains both.
            audit_pack[0] = res

            if res.error_msg:
                print(f"⚠️ Simulation Stopped: {res.error_msg}")
                return

            # ... (Plotting code only runs if res.normalized_plot_data is not empty) ...
            if not res.normalized_plot_data.empty:
                # Plotting
                with fig.batch_update():
                    cols = res.normalized_plot_data.columns.tolist()
                    for i in range(50):
                        if i < len(cols):
                            fig.data[i].update(
                                x=res.normalized_plot_data.index,
                                y=res.normalized_plot_data[cols[i]],
                                name=cols[i],
                                visible=True,
                            )
                        else:
                            fig.data[i].visible = False

                    fig.data[50].update(
                        x=res.benchmark_series.index,
                        y=res.benchmark_series.values,
                        name=f"Benchmark ({inputs.benchmark_ticker})",
                        visible=not res.benchmark_series.empty,
                    )
                    fig.data[51].update(
                        x=res.portfolio_series.index,
                        y=res.portfolio_series.values,
                        visible=True,
                    )

                    # Visual Lines
                    fig.layout.shapes = [
                        dict(
                            type="line",
                            x0=res.decision_date,
                            y0=0,
                            x1=res.decision_date,
                            y1=1,
                            xref="x",
                            yref="paper",
                            line=dict(color="red", width=2, dash="dash"),
                        ),
                        dict(
                            type="line",
                            x0=res.buy_date,
                            y0=0,
                            x1=res.buy_date,
                            y1=1,
                            xref="x",
                            yref="paper",
                            line=dict(color="blue", width=2, dash="dot"),
                        ),
                    ]

                    fig.layout.annotations = [
                        dict(
                            x=res.decision_date,
                            y=0.05,
                            xref="x",
                            yref="paper",
                            text="DECISION",
                            showarrow=False,
                            bgcolor="red",
                            font=dict(color="white"),
                        ),
                        dict(
                            x=res.buy_date,
                            y=1.0,
                            xref="x",
                            yref="paper",
                            text="ENTRY (T+1)",
                            showarrow=False,
                            bgcolor="blue",
                            font=dict(color="white"),
                        ),
                    ]

                start_date = res.start_date.date()
                act_date = res.decision_date.date()
                entry_date = res.buy_date.date()

            # Liquidity Audit Print
            if (
                inputs.mode == "Ranking"
                and res.debug_data
                and "audit_liquidity" in res.debug_data
            ):
                audit = res.debug_data["audit_liquidity"]
                if audit:
                    raw_percentile = audit.get("percentile_setting", 0)
                    keep_pct = (
                        1 - raw_percentile
                    ) * 100  # Calculates the actual portion kept
                    cut_val = audit.get("final_cutoff_usd", 0)

                    print("-" * 60)
                    print(f"🔍 LIQUIDITY CHECK (On Decision Date: {act_date})")
                    print(
                        f"   Universe Size: {audit.get('total_tickers_available')} tickers"
                    )
                    print(
                        f"   Liquidity Threshold: {raw_percentile*100:.0f}th Percentile"
                    )
                    print(f"   Action: Keeping the Top {keep_pct:.0f}% of Market")
                    print(f"   Calculated Cutoff: ${cut_val:,.0f} / day")
                    print(f"   Tickers Remaining: {audit.get('tickers_passed')}")
                    print("-" * 60)

            # --- UPDATED TIMELINE PRINT ---
            print(
                f"Timeline: Start [ {start_date} ] --> Decision [ {act_date} ] --> Cash (1d) --> Entry [ {entry_date} ] --> End [ {res.holding_end_date.date()} ]"
            )

            if inputs.mode == "Ranking":
                print(f"Ranked Tickers ({len(res.tickers)}):")
                for i in range(0, len(res.tickers), 10):
                    print(", ".join(res.tickers[i : i + 10]))
            else:
                print("Manual Portfolio Tickers:")
                for i in range(0, len(res.tickers), 10):
                    print(", ".join(res.tickers[i : i + 10]))

            m = res.perf_metrics

            # --- DRY UI GENERATION ---
            # 1. Define the metrics we want to display
            metrics_to_show = [
                ("Gain", "gain"),
                ("Sharpe", "sharpe"),
                ("Sharpe (ATRP)", "sharpe_atrp"),
                ("Sharpe (TRP)", "sharpe_trp"),  # <--- PINPOINT CHANGE: Add this line
            ]

            rows = []
            for label, key in metrics_to_show:
                p_row = {
                    "Metric": f"Group {label}",
                    "Full": m.get(f"full_p_{key}"),
                    "Lookback": m.get(f"lookback_p_{key}"),
                    "Holding": m.get(f"holding_p_{key}"),
                }
                b_row = {
                    "Metric": f"Benchmark {label}",
                    "Full": m.get(f"full_b_{key}"),
                    "Lookback": m.get(f"lookback_b_{key}"),
                    "Holding": m.get(f"holding_b_{key}"),
                }

                # Delta calculation
                d_row = {"Metric": f"== {label} Delta"}
                for col in ["Full", "Lookback", "Holding"]:
                    d_row[col] = (p_row[col] or 0) - (b_row[col] or 0)

                rows.extend([p_row, b_row, d_row])

            df_report = pd.DataFrame(rows).set_index("Metric")

            # --- 2. STYLING (The "Senior" Design) ---
            # --- 1. PREP DATA (Flattening the Index) ---
            # We convert the index to a column so "Metric" sits on the same row as other headers
            df_report = pd.DataFrame(rows)
            df_report = df_report.set_index("Metric")

            # --- 2. THE STYLING (Sleek & Proportional) ---
            def apply_sleek_style(styler):
                # Match notebook font size (usually 13px)
                styler.format("{:+.4f}", na_rep="N/A")

                # Dynamic Row Highlighting
                def row_logic(row):
                    if "Delta" in row.name:
                        return [
                            "background-color: #f9f9f9; font-weight: 600; border-top: 1px solid #ddd"
                        ] * len(row)
                    if "Group" in row.name:
                        return ["color: #2c5e8f; background-color: #fcfdfe"] * len(row)
                    return ["color: #555"] * len(
                        row
                    )  # Benchmark rows are slightly muted

                styler.apply(row_logic, axis=1)

                styler.set_table_styles(
                    [
                        # Base Table Font - Scaling down to match standard text
                        {
                            "selector": "",
                            "props": [
                                ("font-family", "inherit"),
                                ("font-size", "12px"),
                                ("border-collapse", "collapse"),
                                ("width", "auto"),
                                ("margin-left", "0"),
                            ],
                        },
                        # Header Row - Flattened and Muted
                        {
                            "selector": "th",
                            "props": [
                                ("background-color", "white"),
                                ("color", "#222"),
                                ("font-weight", "600"),
                                ("padding", "6px 12px"),
                                ("border-bottom", "2px solid #444"),
                                ("text-align", "center"),
                                (
                                    "vertical-align",
                                    "bottom",
                                ),  # Aligns 'Metric' with others
                            ],
                        },
                        # Index Column (The "Metric" labels)
                        {
                            "selector": "th.row_heading",
                            "props": [
                                ("text-align", "left"),
                                ("padding-right", "30px"),
                                ("border-bottom", "1px solid #eee"),
                            ],
                        },
                        # Cell Data - Tighter padding
                        {
                            "selector": "td",
                            "props": [
                                ("padding", "4px 12px"),
                                ("border-bottom", "1px solid #eee"),
                            ],
                        },
                        # Remove the extra "Index Name" row completely
                        {
                            "selector": "thead tr:nth-child(1) th",
                            "props": [("display", "table-cell")],
                        },
                    ]
                )

                # Hack to fix the 'Metric' alignment:
                # We remove the index name and set it as the horizontal label for the index
                styler.index.name = None

                return styler

            display(apply_sleek_style(df_report.style))

    update_button.on_click(update_plot)
    update_plot(None)
    display(ui, fig)
    return audit_pack, engine

In [1]:
def plot_walk_forward_analyzer(
    df_ohlcv,
    features_df=None,
    df_close_wide=None,
    df_atrp_wide=None,
    df_trp_wide=None,
    default_start_date="2025-01-17",
    default_lookback=10,
    default_holding=5,
    default_strategy="Sharpe (ATRP)",
    default_rank_start=1,
    default_rank_end=10,
    default_benchmark_ticker=GLOBAL_SETTINGS["benchmark_ticker"],
    master_calendar_ticker=GLOBAL_SETTINGS["calendar_ticker"],
    quality_thresholds=GLOBAL_SETTINGS["thresholds"],
    debug=True,
):

    engine = AlphaEngine(
        df_ohlcv,
        features_df=features_df,
        df_close_wide=df_close_wide,
        df_atrp_wide=df_atrp_wide,
        df_trp_wide=df_trp_wide,  # <--- Update your class to accept this
        master_ticker=master_calendar_ticker,
    )

    # Initialize containers
    audit_pack = [None]  # Unified container

    # If no thresholds passed, use the global Source of Truth
    if quality_thresholds is None:
        quality_thresholds = GLOBAL_SETTINGS["thresholds"]

    # --- Widgets ---
    mode_selector = widgets.RadioButtons(
        options=["Ranking", "Manual List"],
        value="Ranking",
        description="Mode:",
        layout={"width": "max-content"},
        style={"description_width": "initial"},
    )
    lookback_input = widgets.IntText(
        value=default_lookback,
        description="Lookback (Days):",
        layout={"width": "200px"},
        style={"description_width": "initial"},
    )
    decision_date_picker = widgets.DatePicker(
        description="Decision Date:",
        value=pd.to_datetime(default_start_date),
        layout={"width": "auto"},
        style={"description_width": "initial"},
    )
    holding_input = widgets.IntText(
        value=default_holding,
        description="Holding (Days):",
        layout={"width": "200px"},
        style={"description_width": "initial"},
    )
    strategy_dropdown = widgets.Dropdown(
        options=list(METRIC_REGISTRY.keys()),
        value=default_strategy,
        description="Strategy:",
        layout={"width": "220px"},
        style={"description_width": "initial"},
    )
    benchmark_input = widgets.Text(
        value=default_benchmark_ticker,
        description="Benchmark:",
        placeholder="Enter Ticker",
        layout={"width": "180px"},
        style={"description_width": "initial"},
    )
    rank_start_input = widgets.IntText(
        value=default_rank_start,
        description="Rank Start:",
        layout={"width": "150px"},
        style={"description_width": "initial"},
    )
    rank_end_input = widgets.IntText(
        value=default_rank_end,
        description="Rank End:",
        layout={"width": "150px"},
        style={"description_width": "initial"},
    )
    manual_tickers_input = widgets.Textarea(
        value="",
        placeholder="Enter tickers...",
        description="Manual Tickers:",
        layout={"width": "400px", "height": "80px"},
        style={"description_width": "initial"},
    )
    update_button = widgets.Button(description="Run Simulation", button_style="primary")
    ticker_list_output = widgets.Output()

    # --- Layouts ---
    timeline_box = widgets.HBox(
        [lookback_input, decision_date_picker, holding_input],
        layout=widgets.Layout(
            justify_content="space-between",
            border="1px solid #ddd",
            padding="10px",
            margin="5px",
        ),
    )
    strategy_box = widgets.HBox([strategy_dropdown, benchmark_input])
    ranking_box = widgets.HBox([rank_start_input, rank_end_input])

    def on_mode_change(c):
        ranking_box.layout.display = "flex" if c["new"] == "Ranking" else "none"
        manual_tickers_input.layout.display = (
            "none" if c["new"] == "Ranking" else "flex"
        )
        strategy_dropdown.disabled = c["new"] == "Manual List"

    mode_selector.observe(on_mode_change, names="value")
    on_mode_change({"new": mode_selector.value})

    ui = widgets.VBox(
        [
            widgets.HTML(
                "<b>1. Timeline Configuration:</b> (Past <--- Decision ---> Future)"
            ),
            timeline_box,
            widgets.HTML("<b>2. Strategy Settings:</b>"),
            widgets.HBox([mode_selector, strategy_box]),
            ranking_box,
            manual_tickers_input,
            widgets.HTML("<hr>"),
            update_button,
            ticker_list_output,
        ],
        layout=widgets.Layout(margin="10px 0 20px 0"),
    )

    fig = go.FigureWidget()
    fig.update_layout(
        title="Event-Driven Walk-Forward Analysis",
        height=600,
        template="plotly_white",
        hovermode="x unified",
    )
    for i in range(50):
        fig.add_trace(go.Scatter(visible=False, line=dict(width=2)))
    fig.add_trace(
        go.Scatter(
            name="Benchmark",
            visible=True,
            line=dict(color="black", width=3, dash="dash"),
        )
    )
    fig.add_trace(
        go.Scatter(
            name="Group Portfolio", visible=True, line=dict(color="green", width=3)
        )
    )

    # --- Update Logic ---
    def update_plot(b):
        ticker_list_output.clear_output()
        manual_list = [
            t.strip().upper()
            for t in manual_tickers_input.value.split(",")
            if t.strip()
        ]
        decision_date_raw = pd.to_datetime(decision_date_picker.value)

        inputs = EngineInput(
            mode=mode_selector.value,
            start_date=decision_date_raw,
            lookback_period=lookback_input.value,
            holding_period=holding_input.value,
            metric=strategy_dropdown.value,
            benchmark_ticker=benchmark_input.value.strip().upper(),
            rank_start=rank_start_input.value,
            rank_end=rank_end_input.value,
            quality_thresholds=quality_thresholds,
            manual_tickers=manual_list,
            debug=debug,
        )

        ########################################
        with ticker_list_output:
            res = engine.run(inputs)

            # CLEANUP: audit_pack now stores the exact output object.
            # No need to manually map 'debug' or 'results' since res contains both.
            audit_pack[0] = res

            if res.error_msg:
                print(f"⚠️ Simulation Stopped: {res.error_msg}")
                return

            # ... (Plotting code only runs if res.normalized_plot_data is not empty) ...
            if not res.normalized_plot_data.empty:
                # Plotting
                with fig.batch_update():
                    cols = res.normalized_plot_data.columns.tolist()
                    for i in range(50):
                        if i < len(cols):
                            fig.data[i].update(
                                x=res.normalized_plot_data.index,
                                y=res.normalized_plot_data[cols[i]],
                                name=cols[i],
                                visible=True,
                            )
                        else:
                            fig.data[i].visible = False

                    fig.data[50].update(
                        x=res.benchmark_series.index,
                        y=res.benchmark_series.values,
                        name=f"Benchmark ({inputs.benchmark_ticker})",
                        visible=not res.benchmark_series.empty,
                    )
                    fig.data[51].update(
                        x=res.portfolio_series.index,
                        y=res.portfolio_series.values,
                        visible=True,
                    )

                    # Visual Lines
                    fig.layout.shapes = [
                        dict(
                            type="line",
                            x0=res.decision_date,
                            y0=0,
                            x1=res.decision_date,
                            y1=1,
                            xref="x",
                            yref="paper",
                            line=dict(color="red", width=2, dash="dash"),
                        ),
                        dict(
                            type="line",
                            x0=res.buy_date,
                            y0=0,
                            x1=res.buy_date,
                            y1=1,
                            xref="x",
                            yref="paper",
                            line=dict(color="blue", width=2, dash="dot"),
                        ),
                    ]

                    fig.layout.annotations = [
                        dict(
                            x=res.decision_date,
                            y=0.05,
                            xref="x",
                            yref="paper",
                            text="DECISION",
                            showarrow=False,
                            bgcolor="red",
                            font=dict(color="white"),
                        ),
                        dict(
                            x=res.buy_date,
                            y=1.0,
                            xref="x",
                            yref="paper",
                            text="ENTRY (T+1)",
                            showarrow=False,
                            bgcolor="blue",
                            font=dict(color="white"),
                        ),
                    ]

                start_date = res.start_date.date()
                act_date = res.decision_date.date()
                entry_date = res.buy_date.date()

            # Liquidity Audit Print
            if (
                inputs.mode == "Ranking"
                and res.debug_data
                and "audit_liquidity" in res.debug_data
            ):
                audit = res.debug_data["audit_liquidity"]
                if audit:
                    raw_percentile = audit.get("percentile_setting", 0)
                    keep_pct = (
                        1 - raw_percentile
                    ) * 100  # Calculates the actual portion kept
                    cut_val = audit.get("final_cutoff_usd", 0)

                    print("-" * 60)
                    print(f"🔍 LIQUIDITY CHECK (On Decision Date: {act_date})")
                    print(
                        f"   Universe Size: {audit.get('total_tickers_available')} tickers"
                    )
                    print(
                        f"   Liquidity Threshold: {raw_percentile*100:.0f}th Percentile"
                    )
                    print(f"   Action: Keeping the Top {keep_pct:.0f}% of Market")
                    print(f"   Calculated Cutoff: ${cut_val:,.0f} / day")
                    print(f"   Tickers Remaining: {audit.get('tickers_passed')}")
                    print("-" * 60)

            # --- UPDATED TIMELINE PRINT ---
            print(
                f"Timeline: Start [ {start_date} ] --> Decision [ {act_date} ] --> Cash (1d) --> Entry [ {entry_date} ] --> End [ {res.holding_end_date.date()} ]"
            )

            if inputs.mode == "Ranking":
                print(f"Ranked Tickers ({len(res.tickers)}):")
                for i in range(0, len(res.tickers), 10):
                    print(", ".join(res.tickers[i : i + 10]))
            else:
                print("Manual Portfolio Tickers:")
                for i in range(0, len(res.tickers), 10):
                    print(", ".join(res.tickers[i : i + 10]))

            m = res.perf_metrics

            # --- DRY UI GENERATION ---
            # 1. Define the metrics we want to display
            metrics_to_show = [
                ("Gain", "gain"),
                ("Sharpe", "sharpe"),
                ("Sharpe (ATRP)", "sharpe_atrp"),
                ("Sharpe (TRP)", "sharpe_trp"),  # <--- PINPOINT CHANGE: Add this line
            ]

            rows = []
            for label, key in metrics_to_show:
                p_row = {
                    "Metric": f"Group {label}",
                    "Full": m.get(f"full_p_{key}"),
                    "Lookback": m.get(f"lookback_p_{key}"),
                    "Holding": m.get(f"holding_p_{key}"),
                }
                b_row = {
                    "Metric": f"Benchmark {label}",
                    "Full": m.get(f"full_b_{key}"),
                    "Lookback": m.get(f"lookback_b_{key}"),
                    "Holding": m.get(f"holding_b_{key}"),
                }

                # Delta calculation
                d_row = {"Metric": f"== {label} Delta"}
                for col in ["Full", "Lookback", "Holding"]:
                    d_row[col] = (p_row[col] or 0) - (b_row[col] or 0)

                rows.extend([p_row, b_row, d_row])

            df_report = pd.DataFrame(rows).set_index("Metric")

            # --- 2. STYLING (The "Senior" Design) ---
            # --- 1. PREP DATA (Flattening the Index) ---
            # We convert the index to a column so "Metric" sits on the same row as other headers
            df_report = pd.DataFrame(rows)
            df_report = df_report.set_index("Metric")

            # --- 2. THE STYLING (Sleek & Proportional) ---
            def apply_sleek_style(styler):
                # Match notebook font size (usually 13px)
                styler.format("{:+.4f}", na_rep="N/A")

                # Dynamic Row Highlighting
                def row_logic(row):
                    if "Delta" in row.name:
                        return [
                            "background-color: #f9f9f9; font-weight: 600; border-top: 1px solid #ddd"
                        ] * len(row)
                    if "Group" in row.name:
                        return ["color: #2c5e8f; background-color: #fcfdfe"] * len(row)
                    return ["color: #555"] * len(
                        row
                    )  # Benchmark rows are slightly muted

                styler.apply(row_logic, axis=1)

                styler.set_table_styles(
                    [
                        # Base Table Font - Scaling down to match standard text
                        {
                            "selector": "",
                            "props": [
                                ("font-family", "inherit"),
                                ("font-size", "12px"),
                                ("border-collapse", "collapse"),
                                ("width", "auto"),
                                ("margin-left", "0"),
                            ],
                        },
                        # Header Row - Flattened and Muted
                        {
                            "selector": "th",
                            "props": [
                                ("background-color", "white"),
                                ("color", "#222"),
                                ("font-weight", "600"),
                                ("padding", "6px 12px"),
                                ("border-bottom", "2px solid #444"),
                                ("text-align", "center"),
                                (
                                    "vertical-align",
                                    "bottom",
                                ),  # Aligns 'Metric' with others
                            ],
                        },
                        # Index Column (The "Metric" labels)
                        {
                            "selector": "th.row_heading",
                            "props": [
                                ("text-align", "left"),
                                ("padding-right", "30px"),
                                ("border-bottom", "1px solid #eee"),
                            ],
                        },
                        # Cell Data - Tighter padding
                        {
                            "selector": "td",
                            "props": [
                                ("padding", "4px 12px"),
                                ("border-bottom", "1px solid #eee"),
                            ],
                        },
                        # Remove the extra "Index Name" row completely
                        {
                            "selector": "thead tr:nth-child(1) th",
                            "props": [("display", "table-cell")],
                        },
                    ]
                )

                # Hack to fix the 'Metric' alignment:
                # We remove the index name and set it as the horizontal label for the index
                styler.index.name = None

                return styler

            display(apply_sleek_style(df_report.style))

    update_button.on_click(update_plot)
    update_plot(None)
    display(ui, fig)
    return audit_pack  # <--- Return ONLY ONE


#

NameError: name 'GLOBAL_SETTINGS' is not defined

We will replicate the "Kernel Registry" logic locally to ensure no "Black Box" drift:
*   **Gain Kernel:** $(\text{Price}_{\text{end}} / \text{Price}_{\text{start}}) - 1$.
*   **Sharpe Kernel:** $(\text{Mean}(\text{Returns}) / \text{Std}(\text{Returns})) \times \sqrt{252}$.
*   **Sharpe Vol (ATRP/TRP) Kernel:** $\text{Mean}(\text{Returns}) / \text{Mean}(\text{Volatility\_Series})$.
    *   *Note:* It is vital to use the mean of the *sliced* volatility for the specific period.

In [4]:
from typing import Union


class QuantUtils:
    # ... (keep other methods: compute_returns, calculate_gain, calculate_sharpe)

    @staticmethod
    def calculate_sharpe_vol(
        returns: Union[pd.Series, pd.DataFrame],
        vol_data: Union[pd.Series, pd.DataFrame],
    ) -> Union[float, pd.Series]:
        """
        Unified Math for Sharpe(ATR) and Sharpe(TRP).
        Logic: Reward / Risk.
        """
        avg_ret = returns.mean()

        # --- DEFENSIVE LOGIC ---
        # If returns is a DataFrame, we are in RANKING mode (cross-sectional).
        # In this mode, vol_data is expected to be a Series indexed by Ticker.
        # Calling .mean() on it would collapse it to a market-average scalar (The Bug).
        if isinstance(returns, pd.DataFrame) and isinstance(vol_data, pd.Series):
            avg_vol = vol_data
        else:
            # We are in REPORT mode (single portfolio) or raw arrays.
            avg_vol = vol_data.mean()

        res = avg_ret / np.maximum(avg_vol, 1e-8)

        if isinstance(res, (pd.Series, pd.DataFrame)):
            return res.replace([np.inf, -np.inf], np.nan).fillna(0.0)
        return float(res) if np.isfinite(res) else 0.0

In [ ]:
import pandas as pd
import numpy as np


def verify_ranking_integrity():
    """
    🛡️ TRIPWIRE: Prevents 'Momentum Collapse' in Volatility-Adjusted Ranking.
    Ensures that Sharpe(Vol) distinguishes between High-Vol and Low-Vol stocks.
    """
    print("--- 🛡️ Starting Ranking Kernel Audit ---")

    # 1. Setup Mock Universe (2 Tickers, 2 Days)
    # Ticker 'VOLATILE': 10% return, but 10% Volatility
    # Ticker 'STABLE': 2% return, but 1% Volatility (The 'Sharpe' Winner)
    data = {"VOLATILE": [1.0, 1.10], "STABLE": [1.0, 1.02]}  # +10%  # +2%
    df_returns = pd.DataFrame(data).pct_change().dropna()

    # Pre-calculated Mean Volatility per ticker (as provided by Engine Observation)
    vol_series = pd.Series({"VOLATILE": 0.10, "STABLE": 0.01})
    vol_mean = (0.10 + 0.01) / 2
    vol_series = pd.Series({"VOLATILE": 0.055, "STABLE": 0.055})

    print(f"data:\n{data}\n")
    print(f"df_returns:\n{df_returns}\n")
    print(f"vol_series:\n{vol_series}\n")

    # 2. Run Kernel
    results = QuantUtils.calculate_sharpe_vol(df_returns, vol_series)

    print(f"results:\n{results}\n")

    # 3. CALCULATE EXPECTED (Pure Math)
    # Volatile Sharpe: 0.10 / 0.10 = 1.0
    # Stable Sharpe:   0.02 / 0.01 = 2.0

    try:
        # Check A: Diversity. If they are the same, normalization didn't happen.
        if np.isclose(results["VOLATILE"], results["STABLE"]):
            raise AssertionError(
                "RANKING COLLAPSE: Both tickers have the same normalized score."
            )

        # Check B: Direction. STABLE must rank higher than VOLATILE.
        if results["STABLE"] < results["VOLATILE"]:
            # This is exactly what happens when the bug turns it into Momentum
            raise AssertionError(
                f"MOMENTUM REGRESSION: 'STABLE' ({results['STABLE']:.2f}) "
                f"ranked below 'VOLATILE' ({results['VOLATILE']:.2f}). "
                "The denominator was likely collapsed to a market average."
            )

        # Check C: Absolute Precision
        if not np.isclose(results["STABLE"], 2.0):
            raise AssertionError(
                f"MATH ERROR: Expected 2.0 for STABLE, got {results['STABLE']}"
            )

        print(
            "✅ RANKING INTEGRITY PASSED: Volatility normalization is strictly enforced."
        )

    except Exception as e:
        print(f"🔥 KERNEL BREACH: {str(e)}")
        raise e


# Auto-run the new check
verify_ranking_integrity()

--- 🛡️ Starting Ranking Kernel Audit ---
data:
{'VOLATILE': [1.0, 1.1], 'STABLE': [1.0, 1.02]}

df_returns:
   VOLATILE  STABLE
1       0.1    0.02

vol_series:
VOLATILE    0.055
STABLE      0.055
dtype: float64

results:
VOLATILE    1.818182
STABLE      0.363636
dtype: float64

🔥 KERNEL BREACH: MOMENTUM REGRESSION: 'STABLE' (0.36) ranked below 'VOLATILE' (1.82). The denominator was likely collapsed to a market average.


AssertionError: MOMENTUM REGRESSION: 'STABLE' (0.36) ranked below 'VOLATILE' (1.82). The denominator was likely collapsed to a market average.